# DATA PREPARATION

In [1]:
import pyspark
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula, OneHotEncoderEstimator, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline, Model

In [2]:


# Below code is Spark 2+
spark = pyspark.sql.SparkSession.builder.appName('test').getOrCreate()
df = spark.read.csv('raw_files/features_newest.csv', header = True, inferSchema = True)

#df = df.select(col("").alias("label"))
df = df.withColumnRenamed("Rented Bikes", "label")
pdf= df.toPandas()


pdf

,Month,Season,Weekday,Holiday,Daily Weather,Daily Weather (Past),Humidity,Humidity (Past),Windspeed,Windspeed (Past),Apparent Temperature (Avg),Apparent Temperature (Avg) (Past),label,Rented Bikes (Future)
0,January,Winter,Monday,False,partly-cloudy-day,fog,0.88,0.94,1.59,0.55,46.740,36.295,281,279.0
1,January,Winter,Tuesday,False,partly-cloudy-day,partly-cloudy-day,0.86,0.88,2.07,1.59,42.150,46.740,279,274.0
2,January,Winter,Wednesday,False,clear-day,partly-cloudy-day,0.86,0.86,4.13,2.07,45.450,42.150,274,161.0
3,January,Winter,Thursday,False,rain,clear-day,0.87,0.86,3.60,4.13,46.200,45.450,161,270.0
4,January,Winter,Friday,False,partly-cloudy-day,rain,0.81,0.87,7.43,3.60,56.085,46.200,270,62.0
5,January,Winter,Saturday,False,partly-cloudy-day,partly-cloudy-day,0.74,0.81,7.05,7.43,44.615,56.085,62,48.0
6,January,Winter,Sunday,False,clear-day,partly-cloudy-day,0.74,0.74,5.44,7.05,43.385,44.615,48,252.0
7,January,Winter,Monday,False,rain,clear-day,0.83,0.74,6.57,5.44,49.535,43.385,252,311.0
8,January,Winter,Tuesday,False,partly-cloudy-day,rain,0.84,0.83,4.12,6.57,39.870,49.535,311,305.0
9,January,Winter,Wednesday,False,clear-day,partly-cloudy-day,0.76,0.84,5.04,4.12,42.130,39.870,305,264.0


In [57]:
df.printSchema()

root
 |-- Month: string (nullable = true)
 |-- Season: string (nullable = true)
 |-- Weekday: string (nullable = true)
 |-- Holiday: boolean (nullable = true)
 |-- Daily Weather: string (nullable = true)
 |-- Daily Weather (Past): string (nullable = true)
 |-- Humidity: double (nullable = true)
 |-- Humidity (Past): double (nullable = true)
 |-- Windspeed: double (nullable = true)
 |-- Windspeed (Past): double (nullable = true)
 |-- Apparent Temperature (Avg): double (nullable = true)
 |-- Apparent Temperature (Avg) (Past): double (nullable = true)
 |-- label: integer (nullable = true)
 |-- Rented Bikes (Future): integer (nullable = true)



In [58]:
cols = df.columns
categoricalColumns = ["Daily Weather", "Month", "Season", "Weekday", "Daily Weather (Past)"]
NoncategoricalColumns = ["Humidity","Humidity (Past)", "Windspeed","Windspeed (Past)", "Apparent Temperature (Avg)", "Apparent Temperature (Avg) (Past)"]
stages = [] # stages in our Pipeline

 

In [59]:
for categoricalCol in categoricalColumns:
    # Category Indexing with StringIndexer
    # https://weishungchung.com/2017/08/14/stringindexer-transform-fails-when-column-contains-nulls/
    stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol + "Index").setHandleInvalid("skip")
    stages += [stringIndexer]




# Transform all features into a vector using VectorAssembler

#Categorical #NonCategorical
assemblerInputsC = [c + "Index" for c in categoricalColumns] 
assemblerInputsNc = [nc for nc in NoncategoricalColumns]
assemblerInputs =  assemblerInputsC + assemblerInputsNc

assembler = VectorAssembler(inputCols= assemblerInputs , outputCol="features")
stages += [assembler]




partialPipeline = Pipeline().setStages(stages)
pipelineModel = partialPipeline.fit(df)
preppedDataDF = pipelineModel.transform(df)

In [25]:
ssj= preppedDataDF.toPandas()
ssj

,Month,Season,Weekday,Holiday,Daily Weather,Daily Weather (Past),Humidity,Humidity (Past),Windspeed,Windspeed (Past),Apparent Temperature (Avg),Apparent Temperature (Avg) (Past),label,Rented Bikes (Future),Daily WeatherIndex,MonthIndex,SeasonIndex,WeekdayIndex,Daily Weather (Past)Index,features
0,January,Winter,Monday,False,partly-cloudy-day,fog,0.88,0.94,1.59,0.55,46.740,36.295,281,279.0,0.0,0.0,0.0,0.0,3.0,"[0.0, 0.0, 0.0, 0.0, 3.0, 0.88, 0.94, 1.59, 0...."
1,January,Winter,Tuesday,False,partly-cloudy-day,partly-cloudy-day,0.86,0.88,2.07,1.59,42.150,46.740,279,274.0,0.0,0.0,0.0,1.0,0.0,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.86, 0.88, 2.07, 1...."
2,January,Winter,Wednesday,False,clear-day,partly-cloudy-day,0.86,0.86,4.13,2.07,45.450,42.150,274,161.0,1.0,0.0,0.0,2.0,0.0,"[1.0, 0.0, 0.0, 2.0, 0.0, 0.86, 0.86, 4.13, 2...."
3,January,Winter,Thursday,False,rain,clear-day,0.87,0.86,3.60,4.13,46.200,45.450,161,270.0,2.0,0.0,0.0,4.0,1.0,"[2.0, 0.0, 0.0, 4.0, 1.0, 0.87, 0.86, 3.6, 4.1..."
4,January,Winter,Friday,False,partly-cloudy-day,rain,0.81,0.87,7.43,3.60,56.085,46.200,270,62.0,0.0,0.0,0.0,5.0,2.0,"[0.0, 0.0, 0.0, 5.0, 2.0, 0.81, 0.87, 7.43, 3...."
5,January,Winter,Saturday,False,partly-cloudy-day,partly-cloudy-day,0.74,0.81,7.05,7.43,44.615,56.085,62,48.0,0.0,0.0,0.0,3.0,0.0,"[0.0, 0.0, 0.0, 3.0, 0.0, 0.74, 0.81, 7.05, 7...."
6,January,Winter,Sunday,False,clear-day,partly-cloudy-day,0.74,0.74,5.44,7.05,43.385,44.615,48,252.0,1.0,0.0,0.0,6.0,0.0,"[1.0, 0.0, 0.0, 6.0, 0.0, 0.74, 0.74, 5.44, 7...."
7,January,Winter,Monday,False,rain,clear-day,0.83,0.74,6.57,5.44,49.535,43.385,252,311.0,2.0,0.0,0.0,0.0,1.0,"[2.0, 0.0, 0.0, 0.0, 1.0, 0.83, 0.74, 6.57, 5...."
8,January,Winter,Tuesday,False,partly-cloudy-day,rain,0.84,0.83,4.12,6.57,39.870,49.535,311,305.0,0.0,0.0,0.0,1.0,2.0,"[0.0, 0.0, 0.0, 1.0, 2.0, 0.84, 0.83, 4.12, 6...."
9,January,Winter,Wednesday,False,clear-day,partly-cloudy-day,0.76,0.84,5.04,4.12,42.130,39.870,305,264.0,1.0,0.0,0.0,2.0,0.0,"[1.0, 0.0, 0.0, 2.0, 0.0, 0.76, 0.84, 5.04, 4...."


In [60]:
ssj.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1511 entries, 0 to 1510
Data columns (total 20 columns):
Month                                1511 non-null object
Season                               1511 non-null object
Weekday                              1511 non-null object
Holiday                              1511 non-null bool
Daily Weather                        1511 non-null object
Daily Weather (Past)                 1511 non-null object
Humidity                             1511 non-null float64
Humidity (Past)                      1511 non-null float64
Windspeed                            1511 non-null float64
Windspeed (Past)                     1511 non-null float64
Apparent Temperature (Avg)           1511 non-null float64
Apparent Temperature (Avg) (Past)    1511 non-null float64
label                                1511 non-null int32
Rented Bikes (Future)                1510 non-null float64
Daily WeatherIndex                   1511 non-null float64
MonthIndex        

In [39]:
#lrModel = LogisticRegression().fit(preppedDataDF)

In [40]:
#display(lrModel, preppedDataDF, "ROC")

In [61]:
# Keep relevant columns
selectedcols = ["features", "label"]
dataset = preppedDataDF.select(selectedcols)
display(dataset)

DataFrame[features: vector, label: int]

In [62]:
### Randomly split data into training and test sets. set seed for reproducibility
(trainingData, testData) = dataset.randomSplit([0.7, 0.3], seed=100)
print(trainingData.count())
print(testData.count())

1053
458


In [63]:
trainingData.describe().show()

+-------+------------------+
|summary|             label|
+-------+------------------+
|  count|              1053|
|   mean| 251.5394112060779|
| stddev|130.34171093966663|
|    min|                16|
|    max|               775|
+-------+------------------+



In [56]:
trainingData.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(11,[5,6,7,8,9,10...|  333|
|(11,[5,6,7,8,9,10...|  336|
|(11,[5,6,7,8,9,10...|  295|
|(11,[5,6,7,8,9,10...|  575|
|[0.0,0.0,0.0,0.0,...|  345|
|[0.0,0.0,0.0,0.0,...|  269|
|[0.0,0.0,0.0,0.0,...|  360|
|[0.0,0.0,0.0,0.0,...|  326|
|[0.0,0.0,0.0,0.0,...|  281|
|[0.0,0.0,0.0,1.0,...|  294|
|[0.0,0.0,0.0,1.0,...|  341|
|[0.0,0.0,0.0,1.0,...|  376|
|[0.0,0.0,0.0,1.0,...|  372|
|[0.0,0.0,0.0,1.0,...|  308|
|[0.0,0.0,0.0,1.0,...|  311|
|[0.0,0.0,0.0,1.0,...|   43|
|[0.0,0.0,0.0,2.0,...|  360|
|[0.0,0.0,0.0,2.0,...|  298|
|[0.0,0.0,0.0,2.0,...|  263|
|[0.0,0.0,0.0,2.0,...|  310|
+--------------------+-----+
only showing top 20 rows



  #  LINEAR REGRESSION

In [28]:
from pyspark.ml.regression import LinearRegression



lr = LinearRegression(labelCol="label", featuresCol="features",maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(trainingData)

# Print the coefficients and intercept for linear regression
print("Coefficients: %s" % str(lrModel.coefficients))
print("Intercept: %s" % str(lrModel.intercept))

# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()


#RMSE measures the differences between predicted values by the model and the actual values. 
#However, RMSE alone is meaningless until we compare with the actual “MV” value, such as mean, min and max. 
#After such comparison, our RMSE looks pretty good.
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)


# Example :R squared at 0.74 indicates that in our model, approximate 74% of the variability in “MV” can be explained using the model.
print("r2: %f" % trainingSummary.r2)

Coefficients: [-3.3984678874234375,-4.073255230209877,20.331481062819282,-28.230737606317255,-6.618037680524991,-238.07567213727577,105.91350574276221,-3.622160718921554,1.7096083920524405,1.172395086606772,-0.583085763067967]
Intercept: 415.2663671499861
numIterations: 11
objectiveHistory: [0.5000000000000002, 0.4737946234306354, 0.3822548702125597, 0.377568045681078, 0.37607436561166446, 0.37337142311038485, 0.372496209355129, 0.37233686711651576, 0.3723064738990029, 0.3722659122466209, 0.3722263649254216]
+-------------------+
|          residuals|
+-------------------+
| 29.157859542934943|
|  3.070581030522078|
| -29.22946140379446|
|  270.4542440226843|
| 29.100629228628918|
| -48.66327687294779|
|   57.6080045809112|
|  24.87482973292856|
| -33.28005567579714|
|-11.541284375161126|
| 61.781235864099756|
|  90.78747001506576|
|   70.5865088237939|
| 23.665245256681942|
|  35.10673684293215|
|-226.43803970190555|
|  80.09403512020094|
|  48.19124136636384|
|  32.53805909588044|
| 

In [30]:
predictions = lrModel.transform(testData)
predictions.select("label","features","prediction").toPandas().head()


,label,features,prediction
0,315,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.69, 0.75, 10.44, 1...",329.402954
1,303,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.85, 0.81, 4.43, 3....",301.341474
2,362,"[0.0, 0.0, 0.0, 0.0, 2.0, 0.82, 0.93, 3.58, 1....",324.234504
3,34,"[0.0, 0.0, 0.0, 0.0, 2.0, 0.89, 0.93, 2.37, 0....",297.770876
4,279,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.86, 0.88, 2.07, 1....",292.877866


  #  Multinominal logistic regression

In [51]:
from pyspark.ml.classification import LogisticRegression


lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(trainingData)

# Print the coefficients and intercept for multinomial logistic regression
print("Coefficients: \n" + str(lrModel.coefficientMatrix))
print("Intercept: " + str(lrModel.interceptVector))

trainingSummary = lrModel.summary

# Obtain the objective per iteration
objectiveHistory = trainingSummary.objectiveHistory
print("objectiveHistory:")
for objective in objectiveHistory:
    print(objective)

# for multiclass, we can inspect metrics on a per-label basis
print("False positive rate by label:")
for i, rate in enumerate(trainingSummary.falsePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("True positive rate by label:")
for i, rate in enumerate(trainingSummary.truePositiveRateByLabel):
    print("label %d: %s" % (i, rate))

print("Precision by label:")
for i, prec in enumerate(trainingSummary.precisionByLabel):
    print("label %d: %s" % (i, prec))

print("Recall by label:")
for i, rec in enumerate(trainingSummary.recallByLabel):
    print("label %d: %s" % (i, rec))

print("F-measure by label:")
for i, f in enumerate(trainingSummary.fMeasureByLabel()):
    print("label %d: %s" % (i, f))

accuracy = trainingSummary.accuracy
falsePositiveRate = trainingSummary.weightedFalsePositiveRate
truePositiveRate = trainingSummary.weightedTruePositiveRate
fMeasure = trainingSummary.weightedFMeasure()
precision = trainingSummary.weightedPrecision
recall = trainingSummary.weightedRecall
print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Coefficients: 
776 X 11 CSCMatrix

Intercept: [-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,0.4190560224591082,1.3036043410991844,2.299340944771728,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,-1.2313374816819131,1.3036043410991982,-1.2313374816819131,0.4190560224591091,2.299340944771746,0.41905602245910833,2.2993409447716973,1.3036043410991984,0.419056022459109,2.2993409447716946,1.3036043410991722,0.4190560224591092,2.2993409447716973,2.712589389388857,1.3036043410991949,2.7125893893888358,2.712589389388845,2.4960513031062046,2.4960513031062126,1.3036043410991796,2.980447235940315,2.299340944771704,0.4190560224591091,3.1122070025140536,-1.2313374816819131,2.299340944771715,2.7974

label 26: 0.0
label 27: 0.0
label 28: 0.0
label 29: 0.0
label 30: 0.0
label 31: 0.0
label 32: 0.0
label 33: 0.0
label 34: 0.0
label 35: 0.0
label 36: 0.0
label 37: 0.0
label 38: 0.0
label 39: 0.0
label 40: 0.0
label 41: 0.0
label 42: 0.0
label 43: 0.0
label 44: 0.0
label 45: 0.0
label 46: 0.0
label 47: 0.0
label 48: 0.0
label 49: 0.0
label 50: 0.0
label 51: 0.0
label 52: 0.0
label 53: 0.0
label 54: 0.0
label 55: 0.0
label 56: 0.0
label 57: 0.0
label 58: 0.0
label 59: 0.0
label 60: 0.0
label 61: 0.0
label 62: 0.0
label 63: 0.0
label 64: 0.0
label 65: 0.0
label 66: 0.0
label 67: 0.0
label 68: 0.0
label 69: 0.0
label 70: 0.0
label 71: 0.0
label 72: 0.0
label 73: 0.0
label 74: 0.0
label 75: 0.0
label 76: 0.0
label 77: 0.0
label 78: 0.0
label 79: 0.0
label 80: 0.0
label 81: 0.0
label 82: 0.0
label 83: 0.0
label 84: 0.0
label 85: 0.0
label 86: 0.0
label 87: 0.0
label 88: 0.0
label 89: 0.0
label 90: 0.0
label 91: 0.0
label 92: 0.0
label 93: 0.0
label 94: 0.0
label 95: 0.0
label 96: 0.0
label 

Accuracy: 0.011396011396011397
FPR: 0.011396011396011397
TPR: 0.011396011396011397
F-measure: 0.00025681152441715826
Precision: 0.0001298690757380216
Recall: 0.011396011396011397


In [55]:
predictions = lrModel.transform(testData)
predictions.select("label","features","prediction").toPandas()

,label,features,prediction
0,315,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.69, 0.75, 10.44, 1...",312.0
1,303,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.85, 0.81, 4.43, 3....",312.0
2,362,"[0.0, 0.0, 0.0, 0.0, 2.0, 0.82, 0.93, 3.58, 1....",312.0
3,34,"[0.0, 0.0, 0.0, 0.0, 2.0, 0.89, 0.93, 2.37, 0....",312.0
4,279,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.86, 0.88, 2.07, 1....",312.0
5,316,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.88, 0.83, 6.86, 4....",312.0
6,309,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.82, 0.66, 3.8, 10....",312.0
7,376,"[0.0, 0.0, 0.0, 1.0, 2.0, 0.77, 0.86, 7.22, 6....",312.0
8,359,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.76, 0.87, 4.49, 2....",312.0
9,279,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.77, 0.8, 5.95, 2.9...",312.0


  #  ONE VS ALL

In [41]:
from pyspark.ml.classification import LogisticRegression, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# instantiate the base classifier.
lr = LogisticRegression(maxIter=10, tol=1E-6, fitIntercept=True)

# instantiate the One Vs Rest Classifier.
ovr = OneVsRest(classifier=lr)

# train the multiclass model.
ovrModel = ovr.fit(trainingData)

# score the model on test data.
predictions_ova = ovrModel.transform(testData)

# obtain evaluator.
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")

# compute the classification error on test data.
accuracy = evaluator.evaluate(predictions_ova)
print("Test Error = %g" % (1.0 - accuracy))

Py4JJavaError: An error occurred while calling o599616.evaluate.
: java.lang.StackOverflowError
	at scala.collection.generic.Growable$class.loop$1(Growable.scala:52)
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:57)
	at scala.collection.mutable.SetBuilder.$plus$plus$eq(SetBuilder.scala:20)
	at scala.collection.TraversableLike$class.to(TraversableLike.scala:590)
	at scala.collection.AbstractTraversable.to(Traversable.scala:104)
	at scala.collection.TraversableOnce$class.toSet(TraversableOnce.scala:304)
	at scala.collection.AbstractTraversable.toSet(Traversable.scala:104)
	at org.apache.spark.sql.catalyst.trees.TreeNode.containsChild$lzycompute(TreeNode.scala:88)
	at org.apache.spark.sql.catalyst.trees.TreeNode.containsChild(TreeNode.scala:88)
	at org.apache.spark.sql.catalyst.trees.TreeNode.org$apache$spark$sql$catalyst$trees$TreeNode$$mapChild$2(TreeNode.scala:294)
	at org.apache.spark.sql.catalyst.trees.TreeNode$$anonfun$4$$anonfun$apply$13.apply(TreeNode.sc

In [47]:
predictions_ova.show(5)

Py4JJavaError: An error occurred while calling o599615.showString.
: java.lang.StackOverflowError
	at scala.collection.generic.Growable$class.$plus$plus$eq(Growable.scala:59)
	at scala.collection.mutable.MapBuilder.$plus$plus$eq(MapBuilder.scala:25)
	at scala.collection.TraversableLike$class.$plus$plus(TraversableLike.scala:146)
	at scala.collection.AbstractTraversable.$plus$plus(Traversable.scala:104)
	at scala.collection.immutable.HashMap.$plus(HashMap.scala:63)
	at scala.collection.immutable.Map$Map4.updated(Map.scala:179)
	at scala.collection.immutable.Map$Map4.$plus(Map.scala:180)
	at scala.collection.immutable.Map$Map4.$plus(Map.scala:165)
	at scala.collection.mutable.MapBuilder.$plus$eq(MapBuilder.scala:29)
	at scala.collection.mutable.MapBuilder.$plus$eq(MapBuilder.scala:25)
	at scala.collection.generic.Growable$$anonfun$$plus$plus$eq$1.apply(Growable.scala:59)
	at scala.collection.generic.Growable$$anonfun$$plus$plus$eq$1.apply(Growable.scala:59)
	at scala.collection.mutable.R

  #  NAIVE BAYES

In [50]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

# train the model
model = nb.fit(trainingData)

# select example rows to display.
predictions = model.transform(testData)
predictions.show()

# compute accuracy on the test set
evaluator = MulticlassClassificationEvaluator(
                                              metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test set accuracy = " + str(accuracy))

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0,0.0,0.0,0.0,...|  315|[-185.30762468472...|[7.06248810727239...|     233.0|
|[0.0,0.0,0.0,0.0,...|  303|[-111.18560109166...|[1.87934934439414...|     243.0|
|[0.0,0.0,0.0,0.0,...|  362|[-130.70491979746...|[5.12184230078302...|     310.0|
|[0.0,0.0,0.0,0.0,...|   34|[-109.88096733985...|[2.23452029539129...|     310.0|
|[0.0,0.0,0.0,1.0,...|  279|[-122.62264447735...|[2.49528448300104...|     181.0|
|[0.0,0.0,0.0,1.0,...|  316|[-156.91196946196...|[4.78943312472323...|     213.0|
|[0.0,0.0,0.0,1.0,...|  309|[-126.04502550920...|[1.97755414275027...|     175.0|
|[0.0,0.0,0.0,1.0,...|  376|[-150.50797684544...|[1.11941969827516...|     175.0|
|[0.0,0.0,0.0,2.0,...|  359|[-137.63887455680...|[2.39686154523270...|     181.0|
|[0.0,0.0,0.0,2.

# LOGISTIC REGRESSION

In [83]:
# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [84]:
# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

Py4JJavaError: An error occurred while calling o33876.coefficients.
: org.apache.spark.SparkException: Multinomial models contain a matrix of coefficients, use coefficientMatrix instead.
	at org.apache.spark.ml.classification.LogisticRegressionModel.coefficients(LogisticRegression.scala:955)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)


In [67]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [68]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

1.0

In [69]:
evaluator.getMetricName()


'areaUnderROC'

In [71]:
print(lr.explainParams())

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features, current: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label, current: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The

In [72]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Create ParamGrid for Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.01, 0.5, 2.0])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())


In [73]:
# Create 5-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=5)

# Run cross validations
cvModel = cv.fit(trainingData)
# this will likely take a fair amount of time because of the amount of models that we're creating and testing

In [74]:
cvModel

CrossValidatorModel_9c78d4bbbbf1

In [75]:
# Use test set to measure the accuracy of our model on new data
predictions = cvModel.transform(testData)

# cvModel uses the best model found from the Cross Validation
# Evaluate best model
evaluator.evaluate(predictions)

1.0

In [76]:
print('Model Intercept: ', cvModel.bestModel.coefficients)

Py4JJavaError: An error occurred while calling o33711.coefficients.
: org.apache.spark.SparkException: Multinomial models contain a matrix of coefficients, use coefficientMatrix instead.
	at org.apache.spark.ml.classification.LogisticRegressionModel.coefficients(LogisticRegression.scala:955)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:238)
	at java.lang.Thread.run(Thread.java:748)


In [80]:
pred= predictions.toPandas()
pred

,features,Month,Season,Weekday,Holiday,Daily Weather,Daily Weather (Past),Humidity,Humidity (Past),Windspeed,Windspeed (Past),Apparent Temperature (Avg),Apparent Temperature (Avg) (Past),label,Rented Bikes (Future),rawPrediction,probability,prediction
0,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.69, 10.44, 37.795,...",January,Winter,Monday,False,partly-cloudy-day,clear-day,0.69,0.75,10.44,15.29,37.795,46.380,315,341.0,"[-0.7425679673184211, -0.7425679673184211, -0....","[0.00030901465226585594, 0.0003090146522658559...",312.0
1,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.85, 4.43, 32.83, 3...",January,Winter,Monday,False,partly-cloudy-day,clear-day,0.85,0.81,4.43,3.56,32.830,33.035,303,333.0,"[-0.7496787570032616, -0.7496787570032616, -0....","[0.0003014447127305019, 0.0003014447127305019,...",312.0
2,"[0.0, 0.0, 0.0, 0.0, 2.0, 0.82, 3.58, 46.915, ...",January,Winter,Monday,False,partly-cloudy-day,rain,0.82,0.93,3.58,1.28,46.915,43.385,362,376.0,"[-0.7650786415584849, -0.7650786415584849, -0....","[0.0002862771180952761, 0.0002862771180952761,...",312.0
3,"[0.0, 0.0, 0.0, 0.0, 2.0, 0.89, 2.37, 34.925, ...",January,Winter,Monday,True,partly-cloudy-day,rain,0.89,0.93,2.37,0.44,34.925,41.135,34,221.0,"[-0.7642304787182708, -0.7642304787182708, -0....","[0.0002869481522144707, 0.0002869481522144707,...",312.0
4,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.86, 2.07, 42.15000...",January,Winter,Tuesday,False,partly-cloudy-day,partly-cloudy-day,0.86,0.88,2.07,1.59,42.150,46.740,279,274.0,"[-0.7687918774641211, -0.7687918774641211, -0....","[0.00028260460754796034, 0.0002826046075479603...",312.0
5,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.88, 6.86, 51.58, 4...",January,Winter,Tuesday,False,partly-cloudy-day,partly-cloudy-day,0.88,0.83,6.86,4.34,51.580,48.290,316,321.0,"[-0.7852680592910589, -0.7852680592910589, -0....","[0.00026695894866506895, 0.0002669589486650689...",312.0
6,"[0.0, 0.0, 0.0, 1.0, 1.0, 0.82, 3.8, 30.835, 2...",January,Winter,Tuesday,False,partly-cloudy-day,clear-day,0.82,0.66,3.80,10.18,30.835,28.715,309,298.0,"[-0.7402048347938505, -0.7402048347938505, -0....","[0.0003112555150643631, 0.0003112555150643631,...",312.0
7,"[0.0, 0.0, 0.0, 1.0, 2.0, 0.77, 7.22, 35.52500...",January,Winter,Tuesday,False,partly-cloudy-day,rain,0.77,0.86,7.22,6.77,35.525,44.065,376,360.0,"[-0.7521334614718089, -0.7521334614718089, -0....","[0.0002993902437245282, 0.0002993902437245282,...",312.0
8,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.76, 4.49, 45.355, ...",January,Winter,Wednesday,False,partly-cloudy-day,partly-cloudy-day,0.76,0.87,4.49,2.84,45.355,44.925,359,328.0,"[-0.7563037424881852, -0.7563037424881852, -0....","[0.00029522449320277524, 0.0002952244932027752...",312.0
9,"[0.0, 0.0, 0.0, 2.0, 0.0, 0.77, 5.95, 38.91, 4...",January,Winter,Wednesday,False,partly-cloudy-day,partly-cloudy-day,0.77,0.80,5.95,2.90,38.910,44.190,279,296.0,"[-0.753294562083658, -0.753294562083658, -0.75...","[0.00029825198370706316, 0.0002982519837070631...",312.0
